In [1]:
import pandas as pd
from feature_extractor import *
import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Flatten, Conv1D, Dropout, MaxPooling1D, LSTM, Bidirectional, GRU, TimeDistributed
from sklearn.model_selection import train_test_split
import numpy as np
from rdkit.Chem import DataStructs
import matplotlib.pyplot as plt
from tensorflow.python.keras.backend import set_session
from tensorflow.python.keras.models import load_model

Using TensorFlow backend.
/usr/local/anaconda3/envs/servier/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/anaconda3/envs/servier/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/anaconda3/envs/servier/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/anaconda3/en

In [2]:
import tensorflow as tf
graph = tf.get_default_graph()

PreProcessing

In [ ]:
df = pd.read_csv('dataset_single.csv')

In [ ]:
df['P1'].plot.hist()

In [ ]:
arr = np.zeros((0,), dtype=np.int8)

In [ ]:
df['smiles'] = df['smiles'].apply(lambda x: fingerprint_features(x))

In [ ]:
X = df['smiles']
y = df['P1']

In [ ]:
for j,i in enumerate(X):
    DataStructs.ConvertToNumpyArray(X.iloc[j], arr)
    X.iloc[j] = arr

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(X,y,train_size=.6, stratify=y, shuffle=True)

In [ ]:
x_val, x_test, y_val, y_test = train_test_split(x_val, y_val,
                                                    stratify=y_val, 
                                                    test_size=0.1, shuffle=True)

In [ ]:
x_train = np.asarray(x_train)
x_train = np.stack(x_train)
x_train = x_train.reshape((x_train.shape[0],2048,1))
x_train

In [ ]:
x_train.shape

In [ ]:
x_val = np.stack(x_val)
x_val = x_val.reshape((x_val.shape[0],2048,1))

In [ ]:
x_val.shape

In [ ]:
x_test = np.stack(x_test)
x_test = x_test.reshape((x_test.shape[0],2048,1))

In [ ]:
x_test.shape

In [ ]:
y_train = np.array(y_train)
y_val = np.array(y_val)
y_test = np.array(y_test)

In [ ]:
y_train = y_train.reshape(y_train.shape[0],1)
y_val = y_val.reshape(y_val.shape[0],1)
y_test = y_test.reshape(y_test.shape[0],1)

Model Deep Learning

In [ ]:
model = Sequential()

model.add(Conv1D(64, kernel_size=(10), input_shape=(2048,1), activation='relu'))
model.add(MaxPooling1D(10))

model.add(LSTM(20))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss=keras.losses.binary_crossentropy,
              optimizer='adam',#keras.optimizers.SGD(lr=0.01,decay=0.001, momentum=0.9, nesterov=True),
              metrics=['accuracy'])

model.summary()

In [ ]:
model.fit(x_train, y_train, epochs=3, batch_size=64, verbose=1, validation_data=(x_val, y_val))

In [ ]:
model.save('servier.h5')

In [ ]:
_, accuracy = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=0)

In [ ]:
model.predict_classes(x_test)

In [ ]:
accuracy

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

Test avec un fichier CSV

In [3]:
model = load_model('servier.h5')

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [4]:
def prediction_smiles(model, path_smiles):
    smiles_prediction = pd.read_csv(path_smiles, sep=',', header=0)
    arr = np.zeros((0,), dtype=np.int8)
    smiles_prediction['smiles'] = smiles_prediction['smiles'].apply(lambda x: fingerprint_features(x))
    DataStructs.ConvertToNumpyArray(smiles_prediction.iloc[0]['smiles'], arr)
    smiles_prediction.iloc[0]['smiles'] = arr
    arr = np.stack(arr)
    arr = arr.reshape(1,2048,1)
    pred = model.predict_classes(arr)
    return pred

In [6]:
pred = prediction_smiles(model, 'smiles.csv')

In [7]:
int(pred)

1